<a href="https://colab.research.google.com/github/Hugo473/SmartTrip_Planner/blob/main/SmartTrip_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
#############################################################
# --- Agente 1: Buscador de Passeios e Pontos Turpísticos --- #
#############################################################
def agente_buscador(destino, data_inicio, data_fim):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de turismo especializado em planejamento de viagens.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar os principais pontos turísticos, passeios e atrações imperdíveis no destino especificado pelo usuário, assim como os preços médios, considerando o período exato da viagem informado.

        1. Busque as **atrações mais populares, bem avaliadas ou que estejam em alta durante as datas da viagem**, incluindo eventos sazonais, festivais, tours guiados, feiras ou atividades culturais relevantes.
        Dê preferência a atrações que estejam ativas no período informado e que tenham destaque em sites especializados em turismo, blogs de viagem, ou portais confiáveis.

        2. Em relação às **hospedagens**, busque no máximo 3 opções com:
          - Alta demanda ou avaliações positivas
          - Localização estratégica (próxima a pontos turísticos ou transporte)
          - Destaque recente em sites como Booking, TripAdvisor, Airbnb ou Google Reviews

          Para cada hospedagem, informe:
          - Nome
          - Preço médio da diária no período informado
          - Avaliação média (se disponível)
          - Localização (bairro/zona)
          - Breve descrição

        3. **Aeroporto mais próximo**:
          Se a cidade de destino **não tiver aeroporto**, informe:
          - O nome do aeroporto mais próximo
          - A cidade onde ele está localizado
          - A distância aproximada até o destino final
          - As formas comuns de transporte até o destino (ex: ônibus, carro, translado)

        4. **Formato de resposta**:
          - Liste até 5 atrações com nome, tipo, preço médio (se possível) e breve descrição
          - Liste até 3 hospedagens com as informações descritas
          - Adicione um campo final com a informação sobre o aeroporto (se necessário)

        Se alguma atração estiver fechada ou indisponível no período informado, descarte-a e priorize alternativas ativas.
        Sempre considere a **data da viagem e o local informado pelo usuário** como contexto principal para a pesquisa.

        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {destino}\nData de hoje: {data_inicio, data_fim}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [9]:
def agente_planejador(destino, data_inicio, data_fim, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um especialista em roteiros de viagem. Crie um roteiro diário personalizado com base nas atrações fornecidas e nas datas da viagem.
        Sempre que possível, **priorize atrações próximas à hospedagem nos primeiros ou últimos passeios do dia**, para otimizar deslocamentos e conforto.
        Sua resposta deve seguir as diretrizes abaixo:

        1. **Considere o período da viagem** (data de início e fim) para dividir as atrações pelos dias disponíveis agrupando passeios por localização geográfica.
        2. **Agrupe atrações próximas geograficamente** no mesmo dia para facilitar a logística do viajante.
        3. **Equilibre o roteiro**, misturando diferentes tipos de passeios (culturais, naturais, gastronômicos, compras, etc.) para evitar dias cansativos ou monótonos.
        4. Sempre que possível, comece o dia com passeios ao ar livre ou com horários fixos e deixe atividades mais livres ou gastronômicas para o fim do dia.
        5. Se houver eventos com horário específico, organize o restante do dia ao redor deles.

        Retorne o resultado no formato estruturado, como:

        **Dia 1 (DATA):**
        - Manhã: Passeio 1 (nome, tipo, descrição breve, preços)
        - Tarde: Passeio 2
        - Noite: Passeio 3

        ...

        Use criatividade e conhecimento turístico para tornar a viagem inesquecível.

        """,
        description="Agente que planeja passeios",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{destino}\nData de incio: {data_inicio}\nData final: {data_fim}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post



In [10]:
######################################
# --- Agente 3: Redator de Viagem --- #
######################################
def agente_redator(destino, data_inicio, data_fim, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
        Você é um redator de turismo que escreve textos envolventes, informativos e personalizados para viajantes.
        Sua tarefa é transformar o plano de viagem e as informações buscadas em um **texto narrativo atrativo**, que oriente e anime o cliente para sua viagem.

        Utilize os seguintes dados:
        - Destino da viagem
        - Datas de ida e volta
        - Hospedagem recomendada (nome e localização)
        - Lista de atrações e passeios organizados por dia (roteiro diário)
        = Lista de custos por passeio, refeição, transporte. E o total de quanto provalmente pode ser custo dessa viagem. (caso disponível)

        Instruções de estilo e conteúdo:
        1. Comece com uma introdução calorosa e personalizada, apresentando o destino e o período da viagem.
        2. Descreva brevemente a hospedagem, destacando localização e benefícios.
        3. Apresente o roteiro de forma **narrativa e fluida**, como um convite à jornada. Use parágrafos por dia, mencionando o que será feito pela manhã, tarde e noite.
        4. Use linguagem inspiradora e descritiva, com um tom leve, positivo e informativo.
        5. Ao final, inclua uma frase de encerramento incentivando a aproveitar ao máximo a experiência.

        Exemplo de tom desejado:
        > Prepare-se para viver dias inesquecíveis no Rio de Janeiro! De 20 a 24 de julho, seu roteiro está repleto de paisagens icônicas, cultura vibrante e momentos de puro relaxamento à beira-mar...

        Saída esperada: um texto completo e fluido, com cerca de 300 a 500 palavras, que possa ser enviado por e-mail ou PDF ao cliente como “Plano da Sua Viagem”.

            """,
        description="Agente redator de viagem"
    )
    entrada_do_agente_redator = f"Tópico: {destino}\nData de incio: {data_inicio}\nData final: {data_fim}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:

print("🚀 Iniciando o SmartTrip Planner (o seu planejamento de viagens) com 3 Agentes Inteligentes 🚀")

# --- Obter o Tópico do Usuário ---
destino = input("🌎 Para qual DESTINO será a viagem?  ")
data_inicio = input("📆 Qual a DATA DE INÍCIO da viagem (dd/mm/aaaa)? ")
data_fim = input("📆 Qual a DATA DE TÉRMINO da viagem (dd/mm/aaaa)? ")

# Inserir lógica do sistema de agentes ################################################
if not destino or not data_inicio or not data_fim:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o seu roteiro para {destino} entre os dias {data_inicio} e {data_fim}")

    lancamentos_buscados = agente_buscador(destino, data_inicio, data_fim)
    print("\n--- 📝 Resultado do Agente 1 (Hospedagens e Atrações) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(destino, data_inicio, data_fim, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador do seu Roteiro) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(destino, data_inicio, data_fim, plano_de_post)
    print(f"\n--- 📝 Aqui está o seu roteiro planejado para {destino} entre os dias {data_inicio} e {data_fim} ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")



🚀 Iniciando o SmartTrip Planner (o seu planejamento de viagens) com 3 Agentes Inteligentes 🚀
